In [3]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install nltk
!pip install re


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py bdist_wheel for sklearn: started
  Running setup.py bdist_wheel for sklearn: finished with status 'done'
  Stored in directory: C:\Users\sakazi\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py bdist_wheel for nltk: started
  Running setup.py bdist_wheel for nltk: finished with status 'done'
  Stored in directory: C:\Users\sakazi\AppData\Local\pip\Cache\wheels\4b\c8\24\b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Could not find a version that satisfies the requirement re (from versions: )
No matching distribution found for re
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import sklearn
import re, nltk

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report
from sklearn.naive_bayes import MultinomialNB

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
total_dataset = pd.read_csv('train.tsv',sep='\t',encoding='utf-8')


In [12]:
for item in total_dataset.head().values:
  print (item)

['Saleem_Safi'
 'مستقبل کا حال اللہ جانتا ہے اور اُس کے بارے میں حتمی بات نہیں کی جا سکتی۔ بعض اوقات کوئی ایک واقعہ اور حادثہ بھی بڑے سانحات کو جنم دے دیتا ہے۔ یوں بھی ہوتا ہے کہ پہلے قیادت خود جذباتی فضا بنا لیتی ہے اور اس کا ارادہ معاملے کو ایک خاص حد سے آگے بڑھانے کا نہیں ہوتا لیکن پھر ایک وقت ایسا آجاتا ہے کہ وہ خود اس جذباتی فضا اور رائے عامہ کی یرغمال بن جاتی ہے اور نہ چاہتے ہوئے بھی اسے اپنے ارادے کے برخلاف قدم اٹھانا پڑتے ہیں۔ یوں اب بھی یہ خطرہ موجود ہے کہ کوئی مرحلہ ایسا آجائے کہ سیاسی مقاصد کے لئے ڈرامہ بازی کرنے والی بھارتی قیادت اپنی پیدا کردہ رائے عامہ کی یرغمال بن کر مجبوراً جنگ کی طرف چلی جائے۔ یوں پاکستان کے محافظین کو اس حالت کے لئے بھی تیار رہنا چاہئے لیکن جہاں تک بھارتی قیادت کی اس روش کا تعلق ہے تو یہ ڈرامہ بازی ہم اس سے پہلے کئی مرتبہ دیکھ چکے ہیں۔ جنونی ضرور ہے لیکن بھارتی قیادت اتنی بے وقوف نہیں کہ ایٹمی پاکستان سے جنگ کی حماقت کر بیٹھے۔ حالیہ جنگی بخار سے بڑھ کر بخار انڈین پارلیمنٹ پر حملے کے بعد پیدا کیا گیا تھا۔ بھارت نے اس سے بڑی ڈرامے بازی ممبئی حملوں کے

In [19]:
#print shape
print('Saleem Safi :(%s)'%(total_dataset.Author == 'Saleem_Safi').sum() )
print('Hamid Mir :(%s)'%(total_dataset.Author == 'Hamid_Mir').sum() )
print('Altaf_Hassan_Qureshi:(%s)'%(total_dataset.Author == 'Altaf_Hassan_Qureshi').sum())
print('Hassan_Nisar:(%s)'%(total_dataset.Author == 'Hassan_Nisar').sum())



Saleem Safi :(517)
Hamid Mir :(515)
Altaf_Hassan_Qureshi:(278)
Hassan_Nisar:(876)


In [21]:
print(total_dataset.shape)

(2186, 2)


In [22]:
#distribution of Class
total_dataset['Author'].value_counts()

Hassan_Nisar            876
Saleem_Safi             517
Hamid_Mir               515
Altaf_Hassan_Qureshi    278
Name: Author, dtype: int64

In [24]:
total_dataset['Content'][0:5]

0    مستقبل کا حال اللہ جانتا ہے اور اُس کے بارے می...
1    سعودی عرب اور پاکستان کے تعلق کی کئی جہتیں ہیں...
2    گزشتہ سال ملک میں عام انتخابات ہوئے جن میں خیب...
3    فریقین ایک دوسرے کے خلاف ہر حد سے گزر چکے تھے۔...
4    میرے منہ میں خاک لیکن اگر فریقین نے ہوش کے ناخ...
Name: Content, dtype: object

In [25]:
punctuation_marks = ['،','!','”','“','؟',':','۔']

In [26]:
punctuation_marks

['،', '!', '”', '“', '؟', ':', '۔']

In [27]:
def Coloum_preprocessing(content):
    content_clean = re.sub(r"http\S+", "", content)
    cleanText = re.sub(r"[a-zA-Z0-9.@#_:)(-]","",content_clean)
    #p = re.compile(r'<.*?>')
    #cleanText = re.sub(nonalpha,'',tweet_clean).strip()#remove english alphabets
    #cleanText = re.sub(p,'',cleanText).strip()#remove english alphabets
    words = [w for w in cleanText.split() if w not in punctuation_marks]
    #print(words)
    return " ".join(words)

In [28]:
sample = total_dataset.iloc[3]['Content']

In [29]:
print(sample)

فریقین ایک دوسرے کے خلاف ہر حد سے گزر چکے تھے۔ مفاہمت اور مصالحت کی ساری کوششیں ناکام ہو چکی تھیں اور کسی بھی فریق کی طرف سے رتی بھر لچک کا مظاہرہ محال ہو گیا تھا۔ چنانچہ فریق اول (شریف برادران) اور ان کے لئے نرم گوشہ رکھنے والے دیگر عناصر اور پوری قوم کو اِس کی قیمت چکانا پڑی۔ اقتدار سے نکل کر جیلوں میں جانا بھی بڑی قیمت تھی لیکن انسانی اور جذباتی حوالوں سے دیکھا جائے تو محترمہ کلثوم نواز کی ایسی صورت میں رحلت کہ شوہر ساتھ تھے اور نہ بیٹی، یقیناً بہت بڑی قیمت تھی۔ انتخابات سے قبل میاں نواز شریف صاحب، مریم نواز اور ان کو اس راستے پر لگانے والے ان کے مشیر یہ تاثر دے رہے تھے کہ اب کی بار وہ کشتیاں جلا چکے ہیں۔ ہم جیسے لوگ جو جمہور کی بالادستی پر کامل یقین رکھتے ہیں اور اس کے لئے حسبِ استطاعت کوشش کرنے میں بھی لگے رہتے ہیں، بھی اگر چوہدری نثار علی خان اور میاں شہباز شریف کے بیانیے کے حق میں تھے تو اس کی وجہ یہی تھی کہ ہمارے سامنے شریف خاندان کا ماضی تھا۔ ہم جانتے تھے کہ یہ لوگ بہادر تو ہیں لیکن لمبے عرصے کے لئے سختی برداشت نہیں کر سکتے۔ اب کی بار تو ہمیں خاندان اور پارٹی کے اندر دو آرا ک

In [30]:
clean = Coloum_preprocessing(sample)

In [43]:
total_dataset['cleanColoum'] = total_dataset['Content'].apply(Coloum_preprocessing)

In [45]:
labels = total_dataset['Author']
X_train, X_test, y_train, y_test = train_test_split(train_dataset, labels, test_size=0.25, random_state=42)

In [46]:
X_train[['Content','cleanColoum','Author']][0:5]

,Content,cleanColoum,Author
534,وہ ایک ادھورا کام کرنے وطن واپس آئیں۔ ادھورا ...,وہ ایک ادھورا کام کرنے وطن واپس آئیں۔ ادھورا ...,Hamid_Mir
30,سی پیک کو سمجھنے کے لئے بیلٹ اینڈ روڈ منصوبے ج...,سی پیک کو سمجھنے کے لئے بیلٹ اینڈ روڈ منصوبے ج...,Saleem_Safi
1318,محاورے، ضرب الامثال صدیوں پر محیط انسانی تجربا...,محاورے، ضرب الامثال صدیوں پر محیط انسانی تجربا...,Hassan_Nisar
1418,مادر وطن کی مٹی کو اپنے پاک لہو سے سیراب کرنے ...,مادر وطن کی مٹی کو اپنے پاک لہو سے سیراب کرنے ...,Hassan_Nisar
2100,پیدائشی معذور تھیں، جسم جڑا ہوا لیکن ہاتھ نہ ص...,پیدائشی معذور تھیں، جسم جڑا ہوا لیکن ہاتھ نہ ص...,Hassan_Nisar


In [47]:
X_train.shape

(1639, 3)

In [48]:
X_test.shape

(547, 3)

## Experiment 1: Using Count Vectorizer without Stopwords Removal with Unigrams

In [51]:
vectorizer = CountVectorizer()
train_feats = vectorizer.fit_transform(X_train['cleanColoum'])
test_feats = vectorizer.transform(X_test.cleanColoum)

In [52]:
train_feats.shape

(1639, 38181)

In [53]:
features = vectorizer.vocabulary_

In [54]:
features

{'طاق': 18040,
 'گیاجس': 36420,
 'بزنس': 5632,
 'یواے': 38008,
 'تجمل': 7781,
 'ظلمات': 18239,
 'اللسان': 2093,
 'ویہڑے': 27383,
 'کھلائے': 34499,
 'ڈراتا': 31473,
 'بھرپایا': 6826,
 'محاذپر': 22067,
 'لواینڈ': 20850,
 'پہلوکا': 29908,
 'غاصبوں': 18848,
 'وعوامی': 27003,
 'بلہ': 5958,
 'مرام': 22385,
 'انہیںسیدھا': 2808,
 'گلبرگہ': 35825,
 'سزائیں': 15418,
 'اصلاحی': 1484,
 'پشیمان': 28991,
 'ہےپیارے': 37776,
 'استاذالاساتذہ': 1134,
 'عقدہ': 18592,
 'ہڈیوں': 37430,
 'مٹاو': 23989,
 'جلوے': 9660,
 'وجوہات': 26710,
 'ایذا': 4139,
 'آپریٹ': 342,
 'اٹے': 3662,
 'رکشے': 13922,
 'چنوں': 30619,
 'نظریں': 25439,
 'مفارقت': 23162,
 'بیوروکریٹس': 7370,
 'پھندوں': 29700,
 'پارسی': 28051,
 'تاجک': 7610,
 'چارماہ': 30264,
 'سنائے': 15755,
 'پھیلائی': 29829,
 'عناصرکو': 18739,
 'انگڑائی': 2780,
 'کرناٹک': 33001,
 'مارہے': 21654,
 'سواکوئی': 15976,
 'اپناتی': 3695,
 'قاعدہ': 19966,
 'عادتا': 18273,
 'مساجروں': 22620,
 'توبس': 8441,
 'کہلیکن': 34800,
 'حماقتوں': 10742,
 'پریشر': 28908,
 'الرحمانو': 19

In [55]:
train_feats.A[0,450:500]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [94]:
# Using Random Forest

In [56]:
rf_model =  RandomForestClassifier()
rf_model.fit(train_feats,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [57]:
X_test['predictedAuthor'] = rf_model.predict(test_feats)

In [58]:
print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))

0.926873857404022


In [59]:
print(f1_score(X_test['Author'],X_test['predictedAuthor'],average='weighted'))

0.9263305645077982


In [60]:
y_train.unique()

array(['Hamid_Mir', 'Saleem_Safi', 'Hassan_Nisar', 'Altaf_Hassan_Qureshi'],
      dtype=object)

In [61]:
print(classification_report(X_test['Author'],X_test['predictedAuthor']))

                      precision    recall  f1-score   support

Altaf_Hassan_Qureshi       0.98      0.94      0.96        65
           Hamid_Mir       0.87      0.92      0.90       128
        Hassan_Nisar       0.93      0.98      0.96       216
         Saleem_Safi       0.94      0.84      0.89       138

           micro avg       0.93      0.93      0.93       547
           macro avg       0.93      0.92      0.93       547
        weighted avg       0.93      0.93      0.93       547



In [62]:
nb_model = MultinomialNB()
nb_model.fit(train_feats,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [64]:
X_test['predictedAuthor'] = nb_model.predict(test_feats)

In [65]:
print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))

0.9908592321755028


In [66]:
print(f1_score(X_test['Author'],X_test['predictedAuthor'],average='weighted'))

0.9908519279526775


In [42]:
lr = LogisticRegression()
lr.fit(train_feats,y_train)
X_test['predictedAuthor'] = lr.predict(test_feats)
print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))

0.732


C:\Users\Zarmeen\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Experiment 2: Using Count Vectorizer using Max Features without Stopwords Removal with Unigrams

In [67]:
for x in range(500,5000,500):
    print("=============================")
    print("Max Features %d"%x)
    vectorizer = CountVectorizer(max_features=x)
    train_feats = vectorizer.fit_transform(X_train['cleanColoum'])
    test_feats = vectorizer.transform(X_test.cleanColoum)
    print("Using Random Forest")
    rf_model =  RandomForestClassifier()
    rf_model.fit(train_feats,y_train)
    X_test['predictedAuthor'] = rf_model.predict(test_feats)
    print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))
    print("Using Naive Bayes")
    nb_model = MultinomialNB()
    nb_model.fit(train_feats,y_train)
    X_test['predictedAuthor'] = nb_model.predict(test_feats)

    print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))
    
    print("Using Logistic Regression")
    lr = LogisticRegression()
    lr.fit(train_feats,y_train)
    X_test['predictedAuthor'] = lr.predict(test_feats)
    print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))
    

Max Features 500
Using Random Forest
0.9597806215722121
Using Naive Bayes
0.9780621572212066
Using Logistic Regression
0.9945155393053017
Max Features 1000
Using Random Forest
0.9579524680073126
Using Naive Bayes
0.979890310786106
Using Logistic Regression
0.9945155393053017
Max Features 1500
Using Random Forest
0.9616087751371115
Using Naive Bayes
0.9835466179159049
Using Logistic Regression
0.9963436928702011
Max Features 2000
Using Random Forest
0.9634369287020109
Using Naive Bayes
0.9817184643510055
Using Logistic Regression
0.9945155393053017
Max Features 2500
Using Random Forest
0.9652650822669104
Using Naive Bayes
0.9835466179159049
Using Logistic Regression
0.9945155393053017
Max Features 3000
Using Random Forest
0.9597806215722121
Using Naive Bayes
0.9835466179159049
Using Logistic Regression
0.9945155393053017
Max Features 3500
Using Random Forest
0.9414990859232175
Using Naive Bayes
0.9853747714808044
Using Logistic Regression
0.9945155393053017
Max Features 4000
Using Rando

## Experiment with TfIdf

In [69]:
vectorizer = TfidfVectorizer(ngram_range=(1,3),max_features=12000)
train_feats = vectorizer.fit_transform(X_train['cleanColoum'])
test_feats = vectorizer.transform(X_test.cleanColoum)
print("Using Random Forest")
rf_model =  RandomForestClassifier()
rf_model.fit(train_feats,y_train)
X_test['predictedAuthor'] = rf_model.predict(test_feats)
print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))
print("Using Naive Bayes")
nb_model = MultinomialNB()
nb_model.fit(train_feats,y_train)
X_test['predictedAuthor'] = nb_model.predict(test_feats)
print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))

print("Using Logistic Regression")
lr = LogisticRegression()
lr.fit(train_feats,y_train)
X_test['predictedAuthor'] = lr.predict(test_feats)
print(accuracy_score(X_test['Author'],X_test['predictedAuthor']))
    

Using Random Forest
0.9488117001828154
Using Naive Bayes
0.9689213893967094
Using Logistic Regression
0.9872029250457038


In [70]:
vectorizer.vocabulary_

{'منصوبہ': 6856,
 'تاہم': 2175,
 'کچھ نہ': 9809,
 'دشمن': 3545,
 'خلاصہ': 3387,
 'دلا': 3576,
 'سے واقف': 4859,
 'کے بیٹے': 10582,
 'ساتھ زیادتی': 4306,
 'تاریخ میں پہلی': 2164,
 'اعلی پنجاب': 553,
 'کے کچھ': 10862,
 'تبدیل نہیں': 2183,
 'مخلص': 6452,
 'کا احساس': 8925,
 'سازش': 4334,
 'ارشد': 274,
 'سمجھ رہے': 4489,
 'کی جو': 10170,
 'ہے مگر': 11746,
 'صرف یہ': 5157,
 'درج': 3508,
 'کے گھاٹ': 10868,
 'میں کبھی': 7261,
 'انصاف میں': 851,
 'عسکریت پسندی': 5410,
 'ہدایات': 11153,
 'نہیں جانتا کہ': 7628,
 'ہے لہ': 11732,
 'رائےدیں': 3921,
 'ہم بھی': 11193,
 'ہجوم': 11152,
 'گے جس': 11115,
 'میں اتنی': 7039,
 'ئی کی': 99,
 'نظر آتا': 7425,
 'دوستوں نے': 3644,
 'فوری طور': 5704,
 'سے ملاقات ہوئی': 4836,
 'ساٹھ': 4377,
 'پشاور کے': 8443,
 'کرچکے ہیں': 9464,
 'سے مکمل': 4839,
 'اور اے این': 959,
 'گزشتہ انتخابات': 10991,
 'قرار دے دیا': 5836,
 'دیکر': 3826,
 'کے قیام': 10749,
 'ہونے کے برابر': 11441,
 'اور میرے': 1070,
 'کے ساتھ اپنے': 10683,
 'کار': 9115,
 'میں پیپلز': 7248,
 'مارشل': 6264,


In [71]:
X_train.iloc[0]['cleanColoum']

'وہ ایک ادھورا کام کرنے وطن واپس آئیں۔ ادھورا کام یہ تھا کہ وہ اپنی سرزمین پر مرنے آئی تھیں۔ مٹی سے اپنی وفا میں سرخ رنگ بھرنے آئی تھیں۔ محترمہ بے نظیر بھٹو کی شہادت سے دس سال قبل ء میں مارکسٹ دانشور ٹیڈ گرانٹ نے اسپین کے شہر بار سلونا میں منو بھائی کو بتایا کہ بے نظیر بھٹو کو قتل کر دیا جائے گا۔ ٹیڈ گرانٹ اور منو بھائی ایک کانفرنس میں اکٹھے ہوئے تھے۔ دونوں میں بڑی دوستی تھی۔ ٹیڈ گرانٹ بائیں بازو کے اُن دانشوروں میں سے ایک تھے جنہوں نے سب سے پہلے سوویت یونین کے بکھرنے کی پیش گوئی کی تھی۔ منو بھائی نے اپنے دوست سے پوچھا کہ کیا بے نظیر بھٹو کو پاکستان کی سیاست سے باہر کر دیا گیا ہے؟ ٹیڈ گرانٹ نے نفی میں سر ہلایا لیکن اگلے ہی لمحے کہا کہ پاکستان کی مقتدر طاقتوں کو بے نظیر بھٹو کی دوبارہ ضرورت پڑے گی یہ طاقتیں اُنہیں پھر سے استعمال کرنے کی کوشش کریں گی اور پھر اُنہیں قتل کر دیا جائے گا۔ کچھ عرصہ بعد منو بھائی نے ٹیڈ گرانٹ کا یہ تجزیہ بے نظیر صاحبہ کو بتا دیا اور کہا کہ آپ کا جسمانی وجود ختم کیا جا سکتا ہے۔ محترمہ بے نظیر بھٹو نے یہ سُن کر کہا ’’ہاں یہ تو میں بھی جانتی ہوں۔‘‘ محترمہ بے 

In [72]:
train_feats.A[0,450:500]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00567963, 0.        , 0.        , 0.01228662,
       0.        , 0.00618414, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01659366, 0.01783147,
       0.01852327, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [73]:
len(vectorizer.vocabulary_)

12000

In [75]:
!pip install sklearn_crfsuite

  Running setup.py bdist_wheel for tabulate: started
  Running setup.py bdist_wheel for tabulate: finished with status 'done'
  Stored in directory: C:\Users\sakazi\AppData\Local\pip\Cache\wheels\2b\67\89\414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate


  Cache entry deserialization failed, entry ignored
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
